In [1]:
import time
import pdb

import requests, json 

import gmaps
with open('apikey.txt') as f:
    api_key = f.readline().strip()
    f.close
    
gmaps.configure(api_key=api_key)



In [2]:
new_york_coordinates = (40.75, -74.00)
gmaps.figure(center=new_york_coordinates, zoom_level=12)


Figure(layout=FigureLayout(height='420px'))

In [3]:
ratings = {}
addresses = {}
URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# https://developers.google.com/places/web-service/search#TextSearchRequests
MIN_USER_RATINGS = 10
MIN_RATING = 4

# home
location = '40.6875627,-74.0035107'
# grand army plaza
# location = '40.671872,-73.972544'
# union st and 4th ave
# location ='40.677485,-73.983310'
# bay ridge
# location = "40.624468,-74.0487134"

#rankby='prominence'
rankby='distance'
keyword='pizza'
ltype='establishment'
radius='3000'
#https://developers.google.com/places/web-service/supported_types

In [4]:
def get_gmaps_results(api_key, location, **kwargs):

    request_url = URL + '?key=' + api_key
    request_url += '&location=' + location
    for name, val in kwargs.items():
        request_url += '&' + name + '=' + val
    r = requests.get(request_url)
    j = r.json()
    return j

def get_next_page(api_key, next_page_token):
    r = requests.get(URL + '?pagetoken=' + next_page_token +
                        '&key=' + api_key)
    for i in range(10):
        j = r.json()
        if not j['results']: # wait for next page to be available
            time.sleep(5)
            continue
        else:
            return j

def process_results(results):
    global ratings, addresses
    for i, res in enumerate(results): 
        if res['user_ratings_total'] < MIN_USER_RATINGS:
            continue
        if res['rating'] < MIN_RATING:
            continue
        addresses[res['name']] = res['vicinity']
        ratings[res['name']] = res['rating']
        print(res['name'], res['rating'])
    return i+1

In [5]:
def runquery(api_key, location, **kwargs):
    j = get_gmaps_results(api_key, location, **kwargs)
    results = j['results']
    totalresults = 0
    
    totalresults += process_results(results)
    
    while 'next_page_token' in j:
        next_page_token = j['next_page_token']
        time.sleep(5)
        j = get_next_page(api_key, next_page_token)
        results = j['results'] 
        totalresults += process_results(results)
        
    return totalresults

In [6]:
# use either rankby or radius
# j = get_gmaps_results(api_key, location, keyword=keyword, ltype=ltype, radius=radius)
runquery(api_key, location, keyword=keyword, ltype=ltype, rankby=rankby)


The House of Pizza & Calzone 4.6
Francesco's Pizzeria & Trattoria 4.2
Fornino At Pier 6-Open Seasonally from April 4.3
Table 87 4.5
Lucali 4.5
Luzzo's BK 4.6
Brado Thin Crust Pizza 4.5
Layla Jones 4.3
Sal's Pizza Store 4.4
Pronto Pizza 4
My Little Pizzeria 4.4
Brooklyn Pizza Market 4.1
Dellarocco's 4.5
Numero 28 4.3
Giardini Pizza 4.1
Monty Q's 4.1
Via Roma Pizza Bar 4.3
Mark's Red Hook Pizza 4.5
The Longshoreman 4.9
Sottocasa 4.6
Pizza Moto 4.7
Fresh 99 cents pizza 4.3
Big Daddy's Pizza 4.3
Fascati Pizzería 4.2
Forcella Fried Pizza 4.6
Da Vinci 4.3
Grimaldi's Pizzeria 4.1
Juliana's 4.6
2 Bros. Pizza 4.1
Ignazio's 4
Justino's Pizza 4.2
Adrienne's Pizzabar 4.4
Underground Pizza 4.4
Metro Star 4.1
99 Cents Hot Pizza 4
Pranzo Pizza & Pasta 4.1
Front Street Pizza 4.4
PIZZA ETALIA 4.1
LBK pizza 4.6
Table 87 Gowanus 4.6
The Grotto Pizzeria 4.4
Little Italy Pizzeria and Deli 4.2
Pizza Secret 4.5
Pizza Town 4.5
La Villa Pizzeria 4.5
Patsy’s Pizzeria 4.5
Lean Crust Pizza 4.4
Keste Wall Street 4

60

In [7]:
jarray = []
for name, rating in reversed(sorted(ratings.items(), key=lambda item: item[1])):
    print (rating, name, addresses[name])
    jarray.append([name, addresses[name], rating])
    


4.9 The Longshoreman 215 Columbia St, Brooklyn
4.7 Pizza Moto 338 Hamilton Ave, Brooklyn
4.6 Table 87 Gowanus 473 3rd Ave, Brooklyn
4.6 LBK pizza 669 Union St, Brooklyn
4.6 Juliana's 19 Old Fulton St, Brooklyn
4.6 Forcella Fried Pizza 445 Albee Square W, Brooklyn
4.6 Sottocasa 298 Atlantic Ave, Brooklyn
4.6 Luzzo's BK 145 Atlantic Ave, Brooklyn
4.6 The House of Pizza & Calzone 132 Union St, Brooklyn
4.5 Keste Wall Street 77 Fulton St, New York
4.5 Patsy’s Pizzeria 450 Dean St, Brooklyn
4.5 La Villa Pizzeria 261 5th Ave, Brooklyn
4.5 Pizza Town 85 5th Ave, Brooklyn
4.5 Pizza Secret 72 5th Ave, Brooklyn
4.5 Mark's Red Hook Pizza 326 Van Brunt St, Brooklyn
4.5 Dellarocco's 214 Hicks St, Brooklyn Heights
4.5 Brado Thin Crust Pizza 155 Atlantic Ave, Brooklyn
4.5 Lucali 575 Henry St, Brooklyn
4.5 Table 87 87 Atlantic Ave, Brooklyn
4.4 Lean Crust Pizza 737 Fulton St, Brooklyn
4.4 The Grotto Pizzeria 69 New St, New York
4.4 Front Street Pizza 80 Front St, Brooklyn
4.4 Underground Pizza 3 Hanov